In [28]:
# Dependencies

import pandas as pd
import numpy as np
import glob
import os
import timeit                                # To check performance
from datetime import datetime, date, time, timedelta

In [29]:
# Export clean CSV file to master CSV file

tic = timeit.default_timer()                            # Monitor performance

ecobici_rides_df = pd.read_csv("dataset_export_files/ecobici_ride_data_for_sql_import.csv", index_col=None, header=0)

toc = timeit.default_timer()                            # Monitor performance
print(f'Time (in seconds) to export unified CSV file: {round(toc - tic, 2)}')

Time (in seconds) to export unified CSV file: 0.81


In [30]:
for col in ecobici_rides_df.columns:
    print(f'{col}, type: {type(ecobici_rides_df.loc[0, col])}')
    


Genero_Usuario, type: <class 'str'>
Edad_Usuario, type: <class 'numpy.float64'>
Ciclo_Estacion_Retiro, type: <class 'numpy.float64'>
Ciclo_Estacion_Arribo, type: <class 'numpy.int64'>
Usage_Timestamp, type: <class 'str'>
Duration, type: <class 'str'>


In [31]:
ecobici_rides_df_test = ecobici_rides_df.copy()

ecobici_rides_df_test

,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration
0,M,28.0,85.0,85,2010-02-16 12:45:37.427,0 days 00:03:05.267000
1,M,35.0,78.0,63,2010-02-19 13:52:30.700,0 days 00:12:08.983000
2,M,34.0,85.0,23,2010-02-21 14:30:46.390,0 days 00:26:48.800000
3,M,46.0,20.0,17,2010-02-22 16:17:10.470,0 days 00:06:23.507000
4,F,28.0,74.0,46,2010-02-23 18:51:57.057,0 days 00:14:56.160000
...,...,...,...,...,...,...
707034,M,27.0,316.0,54,2021-12-05 09:29:40,0 days 00:13:21
707035,M,49.0,158.0,83,2021-12-05 09:24:30,0 days 00:10:46
707036,M,33.0,417.0,313,2021-12-05 08:55:46,0 days 00:12:03
707037,F,23.0,467.0,260,2021-12-05 08:54:12,0 days 00:33:40


In [50]:
ecobici_rides_df_test['Duration - Minutes'] = pd.Timedelta(ecobici_rides_df_test['Duration']).seconds*60
ecobici_rides_df_test

ValueError: Value must be Timedelta, string, integer, float, timedelta or convertible, not Series

In [49]:
minutes = round(pd.Timedelta(ecobici_rides_df_test.loc[0, 'Duration']).seconds / 60, 1)
minutes

3.1

In [52]:
tic = timeit.default_timer()                            # Monitor performance

for index, row in ecobici_rides_df_test.iterrows():
               
    try: 
        
        ecobici_rides_df_test.at[index, 'Duration - minutes'] = round(pd.Timedelta(ecobici_rides_df_test.loc[index, 'Duration']).seconds / 60, 1)
        
        if index % 100000 == 0: print(f'We\'re up to {index}!')
               
    except: 
        print(f'ERROR: ROW {index} DROPPED')

                                          
toc = timeit.default_timer()                            # Monitor performance
print(f'Time (in seconds) to process data: {round(toc - tic, 2)}')

ecobici_rides_df_test

We're up to 0!
We're up to 100000!
We're up to 200000!
We're up to 300000!
We're up to 400000!
We're up to 500000!
We're up to 600000!
We're up to 700000!
Time (in seconds) to process data: 66.54


,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration,Duration - minutes
0,M,28.0,85.0,85,2010-02-16 12:45:37.427,0 days 00:03:05.267000,3.1
1,M,35.0,78.0,63,2010-02-19 13:52:30.700,0 days 00:12:08.983000,12.1
2,M,34.0,85.0,23,2010-02-21 14:30:46.390,0 days 00:26:48.800000,26.8
3,M,46.0,20.0,17,2010-02-22 16:17:10.470,0 days 00:06:23.507000,6.4
4,F,28.0,74.0,46,2010-02-23 18:51:57.057,0 days 00:14:56.160000,14.9
...,...,...,...,...,...,...,...
707034,M,27.0,316.0,54,2021-12-05 09:29:40,0 days 00:13:21,13.3
707035,M,49.0,158.0,83,2021-12-05 09:24:30,0 days 00:10:46,10.8
707036,M,33.0,417.0,313,2021-12-05 08:55:46,0 days 00:12:03,12.1
707037,F,23.0,467.0,260,2021-12-05 08:54:12,0 days 00:33:40,33.7


In [56]:
ecobici_rides_df_test = ecobici_rides_df_test.rename(columns={"Duration - minutes": "Duration(Min)"})
ecobici_rides_df_test

,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration,Duration(Min)
0,M,28.0,85.0,85,2010-02-16 12:45:37.427,0 days 00:03:05.267000,3.1
1,M,35.0,78.0,63,2010-02-19 13:52:30.700,0 days 00:12:08.983000,12.1
2,M,34.0,85.0,23,2010-02-21 14:30:46.390,0 days 00:26:48.800000,26.8
3,M,46.0,20.0,17,2010-02-22 16:17:10.470,0 days 00:06:23.507000,6.4
4,F,28.0,74.0,46,2010-02-23 18:51:57.057,0 days 00:14:56.160000,14.9
...,...,...,...,...,...,...,...
707034,M,27.0,316.0,54,2021-12-05 09:29:40,0 days 00:13:21,13.3
707035,M,49.0,158.0,83,2021-12-05 09:24:30,0 days 00:10:46,10.8
707036,M,33.0,417.0,313,2021-12-05 08:55:46,0 days 00:12:03,12.1
707037,F,23.0,467.0,260,2021-12-05 08:54:12,0 days 00:33:40,33.7


In [57]:
# Drop 6 columns now that they are empty or of no use. 

tic = timeit.default_timer()                            # Monitor performance

ecobici_rides_df_test.drop(['Duration'], 
                      inplace=True, axis=1)

toc = timeit.default_timer() 
print(f'Processing time (in seconds): {round(toc - tic, 2)}')

ecobici_rides_df_test

Processing time (in seconds): 0.04


,Genero_Usuario,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,Usage_Timestamp,Duration(Min)
0,M,28.0,85.0,85,2010-02-16 12:45:37.427,3.1
1,M,35.0,78.0,63,2010-02-19 13:52:30.700,12.1
2,M,34.0,85.0,23,2010-02-21 14:30:46.390,26.8
3,M,46.0,20.0,17,2010-02-22 16:17:10.470,6.4
4,F,28.0,74.0,46,2010-02-23 18:51:57.057,14.9
...,...,...,...,...,...,...
707034,M,27.0,316.0,54,2021-12-05 09:29:40,13.3
707035,M,49.0,158.0,83,2021-12-05 09:24:30,10.8
707036,M,33.0,417.0,313,2021-12-05 08:55:46,12.1
707037,F,23.0,467.0,260,2021-12-05 08:54:12,33.7


In [55]:
ecobici_rides_df_test['Duration - minutes'].describe()


count    707039.000000
mean         14.433794
std          30.290753
min           0.000000
25%           6.400000
50%          10.300000
75%          16.900000
max        1437.200000
Name: Duration - minutes, dtype: float64

In [ ]:
# Export clean CSV file to master CSV file

tic = timeit.default_timer()                            # Monitor performance

ecobici_rides_df_test2.to_csv("dataset_export_files/ecobici_ride_data_for_sql_import.csv", index=False) 

toc = timeit.default_timer()                            # Monitor performance
print(f'Time (in seconds) to export CSV file: {round(toc - tic, 2)}')

ecobici_rides_df_test2.head()